In [8]:
from google.cloud import bigquery, pubsub_v1
from google.oauth2 import service_account
import requests, json, time, os

project_id = 'persuasive-pipe-384604'
topic_name = 'stock_streaming'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\karth\Documents\key\gcpkey.json"

# Create a client object
client = secretmanager.SecretManagerServiceClient()
secret_id = 'apikey'
version_id = 'latest'  # Use the latest version of the secret
name = f"projects/211234264418/secrets/{secret_id}/versions/{version_id}"
response = client.access_secret_version(request={"name": name})
# Get the secret value
passkey = response.payload.data.decode('UTF-8')


#Create publisher client
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_name)

# Load JSON data

stock_sym = ['AAPL', 'MSFT', 'GOOGL', 'NVDA' ,'META']
while True :
    for symbol in stock_sym:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={passkey}"
        r = requests.get(url)
        stock_data = r.json()
        for timestamp, values in stock_data["Time Series (5min)"].items() :
            row = {
        "symbol" : symbol,
        "timestamp": timestamp,
        "open": float(values["1. open"]),
        "high": float(values["2. high"]),
        "low": float(values["3. low"]),
        "close": float(values["4. close"]),
        "volume": int(values["5. volume"]),
       }
#Publish to topic
            json_string = json.dumps(row)
            print(json_string)
            future = publisher.publish(topic_path, data=json_string.encode('utf-8'))
            print(f"Published message for {symbol} with ID: {future.result()}")
    time.sleep(45)

{"symbol": "AAPL", "timestamp": "2023-04-28 20:00:00", "open": 169.39, "high": 169.59, "low": 169.37, "close": 169.59, "volume": 9054}
Published message for AAPL with ID: 7599507778048999
{"symbol": "AAPL", "timestamp": "2023-04-28 19:55:00", "open": 169.37, "high": 169.38, "low": 169.36, "close": 169.37, "volume": 3560}
Published message for AAPL with ID: 7599519702770420
{"symbol": "AAPL", "timestamp": "2023-04-28 19:50:00", "open": 169.37, "high": 169.38, "low": 169.35, "close": 169.38, "volume": 5011}
Published message for AAPL with ID: 7599530129173299
{"symbol": "AAPL", "timestamp": "2023-04-28 19:45:00", "open": 169.32, "high": 169.41, "low": 169.32, "close": 169.35, "volume": 14284}
Published message for AAPL with ID: 7599507143011022
{"symbol": "AAPL", "timestamp": "2023-04-28 19:40:00", "open": 169.3, "high": 169.33, "low": 169.3, "close": 169.32, "volume": 1666}
Published message for AAPL with ID: 7599531009722230
{"symbol": "AAPL", "timestamp": "2023-04-28 19:35:00", "open"

Published message for AAPL with ID: 7599506152165243
{"symbol": "AAPL", "timestamp": "2023-04-28 16:10:00", "open": 169.65, "high": 169.7, "low": 169.6, "close": 169.6, "volume": 290793}
Published message for AAPL with ID: 7599506802004909
{"symbol": "AAPL", "timestamp": "2023-04-28 16:05:00", "open": 169.68, "high": 169.77, "low": 169.61, "close": 169.65, "volume": 1968160}
Published message for AAPL with ID: 7599519116088779
{"symbol": "AAPL", "timestamp": "2023-04-28 16:00:00", "open": 169.3649, "high": 169.71, "low": 169.34, "close": 169.69, "volume": 2943646}
Published message for AAPL with ID: 7599508187216356
{"symbol": "AAPL", "timestamp": "2023-04-28 15:55:00", "open": 168.94, "high": 169.52, "low": 168.92, "close": 169.3613, "volume": 1189902}
Published message for AAPL with ID: 7599531672373192
{"symbol": "AAPL", "timestamp": "2023-04-28 15:50:00", "open": 169.12, "high": 169.15, "low": 168.86, "close": 168.94, "volume": 584494}
Published message for AAPL with ID: 7599531076

Published message for AAPL with ID: 7599506283990123
{"symbol": "AAPL", "timestamp": "2023-04-28 12:20:00", "open": 168.8691, "high": 168.89, "low": 168.5094, "close": 168.51, "volume": 349301}
Published message for AAPL with ID: 7599499537722188
{"symbol": "AAPL", "timestamp": "2023-04-28 12:15:00", "open": 168.83, "high": 168.9, "low": 168.7786, "close": 168.855, "volume": 280457}
Published message for AAPL with ID: 7599519263628683
{"symbol": "AAPL", "timestamp": "2023-04-28 12:10:00", "open": 168.77, "high": 168.97, "low": 168.76, "close": 168.83, "volume": 375478}
Published message for AAPL with ID: 7599506485106798
{"symbol": "AAPL", "timestamp": "2023-04-28 12:05:00", "open": 168.77, "high": 168.835, "low": 168.69, "close": 168.775, "volume": 267319}
Published message for AAPL with ID: 7599506603489986
{"symbol": "AAPL", "timestamp": "2023-04-28 12:00:00", "open": 168.61, "high": 168.83, "low": 168.57, "close": 168.76, "volume": 383085}
Published message for AAPL with ID: 759953

Published message for MSFT with ID: 7599519116069235
{"symbol": "MSFT", "timestamp": "2023-04-28 16:30:00", "open": 307.4, "high": 307.4399, "low": 307.37, "close": 307.37, "volume": 2051}
Published message for MSFT with ID: 7599537035230350
{"symbol": "MSFT", "timestamp": "2023-04-28 16:25:00", "open": 307.37, "high": 307.54, "low": 307.26, "close": 307.46, "volume": 11170}
Published message for MSFT with ID: 7599536365066512
{"symbol": "MSFT", "timestamp": "2023-04-28 16:20:00", "open": 307.35, "high": 307.5, "low": 307.26, "close": 307.35, "volume": 8315}
Published message for MSFT with ID: 7599536749345125
{"symbol": "MSFT", "timestamp": "2023-04-28 16:15:00", "open": 307.31, "high": 307.37, "low": 307.26, "close": 307.3, "volume": 16122}
Published message for MSFT with ID: 7599506531116983
{"symbol": "MSFT", "timestamp": "2023-04-28 16:10:00", "open": 307.43, "high": 307.43, "low": 307.26, "close": 307.31, "volume": 188046}
Published message for MSFT with ID: 7599508187386969
{"sy

Published message for MSFT with ID: 7599506589149399
{"symbol": "MSFT", "timestamp": "2023-04-28 12:55:00", "open": 304.88, "high": 305.5, "low": 304.88, "close": 304.99, "volume": 241857}
Published message for MSFT with ID: 7599530563653547
{"symbol": "MSFT", "timestamp": "2023-04-28 12:50:00", "open": 304.77, "high": 304.95, "low": 304.51, "close": 304.88, "volume": 199773}
Published message for MSFT with ID: 7599507778183010
{"symbol": "MSFT", "timestamp": "2023-04-28 12:45:00", "open": 304.215, "high": 304.7494, "low": 304.18, "close": 304.7473, "volume": 227641}
Published message for MSFT with ID: 7599507692153555
{"symbol": "MSFT", "timestamp": "2023-04-28 12:40:00", "open": 304.605, "high": 304.66, "low": 304.1, "close": 304.24, "volume": 278214}
Published message for MSFT with ID: 7599537159648087
{"symbol": "MSFT", "timestamp": "2023-04-28 12:35:00", "open": 304.74, "high": 305.04, "low": 304.47, "close": 304.605, "volume": 286769}
Published message for MSFT with ID: 759950714

...

Published message for NVDA with ID: 7599530544816075
{"symbol": "NVDA", "timestamp": "2023-04-28 14:15:00", "open": 275.74, "high": 275.8343, "low": 275.4, "close": 275.4857, "volume": 195551}
Published message for NVDA with ID: 7599507447144831
{"symbol": "NVDA", "timestamp": "2023-04-28 14:10:00", "open": 275.84, "high": 275.8791, "low": 275.595, "close": 275.76, "volume": 165736}
Published message for NVDA with ID: 7599531672491449
{"symbol": "NVDA", "timestamp": "2023-04-28 14:05:00", "open": 275.355, "high": 275.93, "low": 275.32, "close": 275.81, "volume": 239252}
Published message for NVDA with ID: 7599537161648136
{"symbol": "NVDA", "timestamp": "2023-04-28 14:00:00", "open": 275.71, "high": 275.71, "low": 275.3081, "close": 275.36, "volume": 271257}
Published message for NVDA with ID: 7599519702851313
{"symbol": "NVDA", "timestamp": "2023-04-28 13:55:00", "open": 275.69, "high": 275.87, "low": 275.44, "close": 275.7193, "volume": 195434}
Published message for NVDA with ID: 759

Published message for META with ID: 7599506297738422
{"symbol": "META", "timestamp": "2023-04-28 19:05:00", "open": 239.4603, "high": 239.4603, "low": 239.46, "close": 239.46, "volume": 494}
Published message for META with ID: 7599507793141353
{"symbol": "META", "timestamp": "2023-04-28 19:00:00", "open": 239.54, "high": 239.55, "low": 239.54, "close": 239.54, "volume": 581}
Published message for META with ID: 7599531687065090
{"symbol": "META", "timestamp": "2023-04-28 18:55:00", "open": 239.55, "high": 239.56, "low": 239.54, "close": 239.56, "volume": 1004}
Published message for META with ID: 7599536375892938
{"symbol": "META", "timestamp": "2023-04-28 18:50:00", "open": 239.48, "high": 239.54, "low": 239.48, "close": 239.54, "volume": 881}
Published message for META with ID: 7599536526779199
{"symbol": "META", "timestamp": "2023-04-28 18:45:00", "open": 239.53, "high": 239.53, "low": 239.53, "close": 239.53, "volume": 200}
Published message for META with ID: 7599507143092786
{"symbo

Published message for META with ID: 7599536645090679
{"symbol": "META", "timestamp": "2023-04-28 15:10:00", "open": 239.9799, "high": 239.9799, "low": 239.48, "close": 239.595, "volume": 374225}
Published message for META with ID: 7599506182858932
{"symbol": "META", "timestamp": "2023-04-28 15:05:00", "open": 240.29, "high": 240.3042, "low": 239.78, "close": 239.96, "volume": 423662}
Published message for META with ID: 7599508187308508
{"symbol": "META", "timestamp": "2023-04-28 15:00:00", "open": 240.15, "high": 240.43, "low": 240.1318, "close": 240.3, "volume": 380273}
Published message for META with ID: 7599536155568663
{"symbol": "META", "timestamp": "2023-04-28 14:55:00", "open": 240.33, "high": 240.39, "low": 240.08, "close": 240.16, "volume": 318636}
Published message for META with ID: 7599506731271845
{"symbol": "META", "timestamp": "2023-04-28 14:50:00", "open": 239.92, "high": 240.38, "low": 239.77, "close": 240.335, "volume": 649805}
Published message for META with ID: 75995

Published message for META with ID: 7599536318917824


KeyboardInterrupt: 

In [4]:
from google.cloud import bigquery, pubsub_v1
import os
from google.oauth2 import service_account
from google.cloud import secretmanager
import requests, json, time

project_id = 'persuasive-pipe-384604'
creds_path = r"C:\Users\karth\Documents\key\gcpkey.json"
topic_name = 'stock_streaming'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\karth\Documents\key\gcpkey.json"
# Authenticate to GCP
# credentials = service_account.Credentials.from_service_account_file(
#     creds_path, scopes=["https://www.googleapis.com/auth/cloud-platform"]
# )

# Create a client object
client = secretmanager.SecretManagerServiceClient()
secret_id = 'apikey'
version_id = 'latest'  # Use the latest version of the secret
name = f"projects/211234264418/secrets/{secret_id}/versions/{version_id}"
response = client.access_secret_version(request={"name": name})
# Get the secret value
passkey = response.payload.data.decode('UTF-8')
print(passkey)


AHB6D8YXN4Z2854M


In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =